In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
keras = tf.keras
from datetime import datetime

from typing import List

import os
import glob

from src.io.load import DataLoader

from dataclasses import dataclass, field
from typing import Dict
from src.action_strategy.factory import ActionStrategy
from src.action_strategy.mean_reversion import MeanReversion
from src.forecast.factory import ModelForecastFactory
from src.forecast.runner import ForecastRunner
from src.forecast.utils import SamplerSetForecast
from src.io.load import DataLoader, load_config
from src.io.save import Dumper
from src.preprocessing.preprocess import ProcessData
from src.stock import Balance, Stock
import logging
from src.trade_manager import TradeManagerV0
from datetime import datetime
import math
from abc import ABC, abstractmethod

In [3]:
PATH_DATA_PROCESSED = "../data/processed/"

In [4]:
import logging
from src.main import load_config, load_data, Experiment

In [5]:
logging.basicConfig()
log=logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

In [6]:
config_file = "1"
# Load config and data
logging.info("Load data and config..")
config_exp = load_config(config_file)
prices = load_data(config_exp["market_name"])
config_exp["date_start"] =  datetime.strptime(' '.join([config_exp["date_start"], config_exp["hour_begin_trade"]]), "%d-%m-%Y %H")
config_exp["date_end"] =  datetime.strptime(' '.join([config_exp["date_end"], config_exp["hour_begin_trade"]]), "%d-%m-%Y %H")

n_days = (config_exp["date_end"] - config_exp["date_start"]).days
config_exp["n_iteration"] = math.ceil(n_days/config_exp["n_frames"])

logging.info("Process data ..")
prices = ProcessData(df=prices).run_preprocess()
prices = prices.to_frame("value")


# Init classes
logging.info("Init classes..")

stock = Stock(storage_pwr=config_exp["storage_pwr"], 
                rho_d=config_exp["rho_d"], 
                rho_s=config_exp["rho_s"],
                t_discharge=config_exp["t_discharge"],
                init_storage_cpty=config_exp["init_storage_cpty"],
                )

balance = Balance(init_level=config_exp["balance_init_level"])

# sampler for train and test sets
sampler = SamplerSetForecast(df=prices, 
                            n_frames=config_exp["n_frames"],
                            )
# init forecast model
model_forecast = ModelForecastFactory().get_model(**{
                                        "name":"arima",
                                        "order":dict(p=24, d=1, q=0),
                                        "seasonal_order":None,
                                        })
logging.info("Init forecast runner..")

# forecast prices run
forecast_runner = ForecastRunner(model=model_forecast,
                                n_frames=config_exp["n_frames"],
                                sampler=sampler,
                                )    
# trading strategy
strategy = MeanReversion(prices=prices,
                        period=config_exp["period"],
                        alpha=config_exp["alpha"],
                        )

exp = Experiment(stock=stock, 
                df=sampler.df,
                forecast_runner=forecast_runner,
                strategy=strategy,
                balance=balance,
                config_exp=config_exp,
                )

# run experience
exp.run()

17557


INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src.trade_manager:Current cpty : 0
INFO:src.trade_manager:Current balance : 1000.0
INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src.trade_manager:Current cpty : 0
INFO:src.trade_manager:Current balance : 1000.0
INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src.trade_manager:Current cpty : 0
INFO:src.trade_manager:Current balance : 1000.0
INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src.trade_manager:Current cpty : 0
INFO:src.trade_manager:Current balance : 1000.0
INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src.trade_manager:Current cpty : 0
INFO:src.trade_manager:Current balance : 1000.0
INFO:src.stock:========
INFO:src.stock:action Stock : HOLD
INFO:src.stock:Updated Stock : 0
INFO:src